## ANT COLONY OPTIMIZATION

In [1]:
import pandas as pd
import numpy as np
import random
import math



KeyboardInterrupt: 

In [2]:
df=pd.read_csv('n50.csv',parse_dates=['Date'],index_col='Date')  #Importing Dataset
df = df.loc["2016-01-01" : ]   #Since 2016-01-01, 5y(1234rows till 2020-12-31)
tdf=df.copy()                  #deep copy
df.reset_index(drop=True, inplace=True)

In [3]:
def number_of_years(y):        #calculates the number of years of the dataset
  p=y.index[0]                 #date of first row in the dataset (datetime format)
  q=y.index[len(y)-1]          #date of last row in the dataset  (datetime format)
  return ((q-p).days+1)/365           #the difference give the number of total days (not trading days) over the total number of years in the dataset

In [4]:
trading_days=len(df)/number_of_years(tdf)                       #Trading days per year (automated)

In [5]:
returnsh=df.pct_change()                  #Here, returnsh would mean return considered for sharpe ratio
returnsh.fillna(0,inplace=True)           #calculating daily returns of the stocks in the portfolio

In [6]:
returnso=returnsh.copy()                  #this cell considers only NEGATIVE returns so as to calculate sortino ratio
for cols in returnso.columns.tolist():
    for i in range(0,len(df)):
        if returnso[cols][i] > 0:
            returnso[cols][i]=0   
            #Here, returnso would mean return considered for sortino ratio

In [7]:
covmatsh=returnsh.cov()*trading_days      #Annualised covariance matrix calculated wrt returnsh i.e. used to calculate sharpe ratio
covmatso=returnso.cov()*trading_days      #Annualised covariance matrix calculated wrt returnso i.e. used to calculate sortino ratio

In [8]:
risk_free_rate = 0.0358                   #initializing risk free rate that will be used in calculating both the ratios (absolute value)
#referred from url: https://www.rbi.org.in/Scripts/BS_NSDPDisplay.aspx?param=4&Id=24292
#In the above url, the 364 (1 year) day treasury bill is 3.58% , when taken absolute value => 0.0358
# (improved)

In [9]:
df
stocks=df.shape[1]
stocks

22

In [10]:
global_warr_sortino=[]
global_war_sharpe=[]

In [11]:
#No of times we want to run the algorithm
#ITERATIONS=500      #Search for Indian Market. IMPORTANT (Perform Trials and look for optimal value)#$#$#$#$#$#$#$#

ITERATIONS=20

#Constant<Fixed amount pheromone>
Q=0.1
#Evaporation rate
EVA_RATE=0.1       #Search for Indian Market. IMPORTANT (Perform Trials and look for optimal value)#$#$#$#$#$#$#$#
#No of ants
#ANTS=168            #Search for Indian Market. IMPORTANT (Perform Trials and look for optimal value)#$#$#$#$#$#$#$#
ANTS=50
#Ant Colony Optimization Approach to Portfolio Optimization – A Lingo Companion Kambiz Forqandoost Haqiqi and Tohid Kazemi

In [12]:
def BALANCE(weights):
    #Making sure the total sum of the weights eual to 1
    weights = [w/sum(weights) for w in weights] # Making sure all weights represent proportions that add up to 1
    return weights

In [13]:
def ratio(a,b,c):                         #function to calculate ratio i.e. "(returns-(risk_free_rate))/deviation"
    #calculating sharpe ratio
    return (a-c)/b                          #a => annual return, b =>deviation (standard for sharpe, semi for sortino) , c =>risk_free_rate 

In [14]:
#Initializing pbest(the best fitness value   SHARPE)
pbest=-1
#Initializing the current fitness value
fitness=0

#for each iteration
for iteration in range(ITERATIONS):
    
    #PREPARAING THE PHEROMONE MATRIX WHERE THE COLS=STOCKS AND  ROWS=ANTS
    pheromon=[[0]*stocks for i in range(ANTS+1)]#why (ants+1)?The last ant can update the pheromone values in the last row
    
    # Initializing the pheromone status 
    for i in range(len(pheromon[0])):
        pheromon[0][i]=random.randint(1,15)   #When input stocks varies, this needs to vary accordingly.(Divide number of stocks / 2)
    
    #copying the values and storing it in temp_pher
    temp_pher=pheromon[0]
    
    #Making sure that the total amount of pheromone equals 1 
    weights=np.array(BALANCE(temp_pher))
        
    #calculating annulaised portfolio return
    returns_temp = np.sum(returnsh.mean()*weights)*trading_days 
    #print(returns_temp)
    
    #calculating portfolio varience wrt calculating sharpe ratio
    varsh=np.dot(weights.T,np.dot(covmatsh,weights)) 
    #print(varsh)
    
    #portfolio risk
    volatility_temp = np.sqrt(varsh)      
    
    #Calculating fitness value(ie sharpe ratio)
    fitness = ratio(returns_temp,volatility_temp,risk_free_rate)
    
    #Initializing the intial fitness value as the best fitness value(pbest)
    if pbest==-1:
         pbest=fitness
    
    #list
    path=[]
   
    #for each ant
    for ant in range(ANTS-1):
        
        #find the total pheromone 
        #print("Pheronome: ",pheromon[ant])
        total=sum(pheromon[ant])
        #print("Total: ",total)
    
        #Initializing probability
        probability=pheromon[ant][:]
        #print("Probability: ",probability)
        
        #finding probability of each stocks pheromone 
        for p in range(len(probability)):
            probability[p]=(probability[p]/total)
            
        #Trying to select stocks in decreasing order based on their pheromone level and storing the stock order in a list(path)
        for stock in range(stocks):
            select=probability.index(max(probability))
            probability[select]=-math.inf
            path.append(select)
            #print("Path: ",path)
            
        
        #Updating the pheromone level of each stock for the next ant 
        #Formula: old pheromone level * (1-eva_rate) + Q * (fitness/pbest) where Q is fixed amount of pheromone
        for s in path:
            pheromon[ant+1][s]=pheromon[ant][s]*(1-EVA_RATE)+Q*(fitness/pbest)
        
        
        #making sure that the updated pheromon adds upto 1
        temp_pher=pheromon[ant+1]
        weights=np.array(BALANCE(temp_pher))
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
        varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
        volatility_temp = np.sqrt(varsh)                              #portfolio risk
        fitness = ratio(returns_temp,volatility_temp,risk_free_rate)  #calculating sharpe ratio
        #comparing the old fitness value with the updated fitness value
        if(fitness>pbest):
            
            #if the updated fitness value is better than the previous, change pbest to present fitness value
            pbest=fitness
            print(pbest)
            #remembering the weights of the best portfolio
            global_warr_sharpe=weights.tolist()
        #print("Global warr sharpe: ",global_warr_sharpe)


1.2254255679559707
1.2268213031415944
1.2283152786302292
1.2299089498717979
1.231602759752605
1.2333959887303465
1.2352866175902726
1.2372712115006863
1.2393448345643345
1.241501003950222
1.2437316917974088
1.2460273813173115
1.2483771808835298
1.2507689965038262
1.2531897591575651
1.2556256994069044
1.2580626578807026
1.2604864171346335
1.2628830384054803
1.2652391861771068
1.267542424361754
1.2697814701785084
1.2719463952040293
1.2740287671665356
1.276021730375404
1.2779200267605724
1.2797199629538185
1.281419331419431
1.283017295220618
1.28451424661068
1.285911649391315
1.2872118740776821
1.2884180335707927
1.2895338254747064
1.290563385589849
1.2915111556017906
1.2923817666576054
1.2931799394273131
1.2939104003994342
1.2945778135458403
1.2951867260849643
1.295741526839026
1.2962464155835105
1.2967053817907654
1.2971221912435946
1.2975003791138697
1.2978432482453797
1.298153871534607
1.2984350974571544
1.2984888123219036
1.2987424231640503
1.29900270665449
1.299267870918942
1.299535

In [15]:
names=df.columns
j=0
for i in (names):
    print("{} -> {} %".format(i,global_warr_sharpe[j]*100))
    j+=1

ASIANPAINT -> 3.387035264362897 %
BAJFINANCE -> 7.265221553234944 %
BAJAJFINSV -> 3.9410618770589037 %
BRITANNIA -> 4.49508848975491 %
DIVISLAB -> 2.278982038970884 %
HCLTECH -> 3.387035264362897 %
HDFCBANK -> 3.9410618770589037 %
HINDALCO -> 2.83300865166689 %
HINDUNILVR -> 7.265221553234944 %
HDFC -> 2.83300865166689 %
ICICIBANK -> 0.6169022008828642 %
INFY -> 3.387035264362897 %
JSWSTEEL -> 6.157168327842929 %
KOTAKBANK -> 0.6169022008828642 %
NESTLEIND -> 5.049115102450917 %
RELIANCE -> 8.373274778626955 %
SHREECEM -> 6.711194940538936 %
TCS -> 3.387035264362897 %
TATASTEEL -> 3.9410618770589037 %
TECHM -> 8.373274778626955 %
TITAN -> 5.603141715146923 %
WIPRO -> 6.157168327842929 %


In [16]:
weights=np.array(global_warr_sharpe)
returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
varsh=np.dot(weights.T,np.dot(covmatsh,weights))              #calculating portfolio varience wrt calculating sharpe ratio
volatility_temp = np.sqrt(varsh)                              #portfolio risk
fitness = ratio(returns_temp,volatility_temp,risk_free_rate)  #calculating sharpe ratio
print("Sharpe Ratio: ",fitness)

Sharpe Ratio:  1.3491064793473246


In [17]:
#Initializing pbest(the best fitness value    SORTINO)
pbest=-1
#Initializing the current fitness value
fitness=0

#for each iteration
for iteration in range(ITERATIONS):
    
    #PREPARAING THE PREROMON MATRIX WHERE THE COLS=STOCKS AND  ROWS=ANTS
    pheromon=[[0]*stocks for i in range(ANTS)]
    for i in range(len(pheromon[0])):
        pheromon[0][i]=random.randint(1,15)

    temp_pher=pheromon[0]
    weights=np.array(BALANCE(temp_pher))
    returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
    varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sortino ratio
    semi_temp = np.sqrt(varso)                                    #portfolio semi-deviation
    fitness = ratio(returns_temp,semi_temp,risk_free_rate)  #calculating Sortino ratio
    if pbest==-1:
         pbest=fitness
    
    path=[]
   
    
   
    for ant in range(ANTS-1):
        total=sum(pheromon[ant])
        probability=pheromon[ant][:]
        for p in range(len(probability)):
            probability[p]=(probability[p]/total)
        for stock in range(stocks):
            select=probability.index(max(probability))
            probability[select]=-math.inf
            path.append(select)
        
            
        for s in path:
            pheromon[ant+1][s]=pheromon[ant][s]*(1-EVA_RATE)+Q*(fitness/pbest)
            
        temp_pher=pheromon[ant+1]
        weights=np.array(BALANCE(temp_pher))
        returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
        varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sortino ratio
        semi_temp = np.sqrt(varso)                                    #portfolio semi-deviation
        fitness = ratio(returns_temp,semi_temp,risk_free_rate)  #calculating Sortino ratio
        if(fitness>pbest):
            pbest=fitness
            #print(pbest)
            global_warr_sortino=weights.tolist()

0.2701457630710407
0.2657747371427953
0.27548706770547987
0.28035219783242016
0.2911787692648987
0.2891254929362958
0.2740494695332606
0.2837055632366366
0.28631119271128364
0.2800245320550164
0.2796224101596918
0.2762691973104403
0.2665306831265507
0.26403144823288005
0.26286713180807736
0.2755529942244773
0.30351606948595555
0.2859383145106997
0.2675744327317647
0.2826856335174921


In [18]:

names=df.columns
j=0
for i in (names):
    print("{} -> {} %".format(i,global_warr_sortino[j]*100))
    j+=1

ASIANPAINT -> 3.052584047600274 %
BAJFINANCE -> 9.02406603901736 %
BAJAJFINSV -> 6.635473242450525 %
BRITANNIA -> 1.2611394501751478 %
DIVISLAB -> 7.829769640733944 %
HCLTECH -> 1.2611394501751478 %
HDFCBANK -> 1.8582876493168567 %
HINDALCO -> 7.829769640733944 %
HINDUNILVR -> 3.649732246741983 %
HDFC -> 0.6639912510334394 %
ICICIBANK -> 3.649732246741983 %
INFY -> 1.8582876493168567 %
JSWSTEEL -> 5.441176844167108 %
KOTAKBANK -> 7.232621441592234 %
NESTLEIND -> 3.649732246741983 %
RELIANCE -> 3.649732246741983 %
SHREECEM -> 3.649732246741983 %
TCS -> 4.8440286450254 %
TATASTEEL -> 8.426917839875651 %
TECHM -> 5.441176844167108 %
TITAN -> 2.4554358484585657 %
WIPRO -> 6.635473242450525 %


In [19]:
weights=np.array(global_warr_sortino)
returns_temp = np.sum(returnsh.mean()*weights)*trading_days   #calculating annulaised portfolio return
varso=np.dot(weights.T,np.dot(covmatso,weights))              #calculating portfolio varience wrt calculating sortino ratio
semi_temp = np.sqrt(varso)                                    #portfolio semi-deviation
fitness = ratio(returns_temp,semi_temp,risk_free_rate)        #calculating Sortino ratio
print("Sortino Ratio: ",fitness)

Sortino Ratio:  2.135681157724758
